# CI/CD

In [2]:
import boto3
import json
import pandas as pd
import numpy as np
from datetime import datetime
import sagemaker
from sagemaker import get_execution_role, Session
from sagemaker.inputs import TrainingInput
from sagemaker.model import Model
from sagemaker.processing import ProcessingInput, ProcessingOutput, ScriptProcessor
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import ProcessingStep, TrainingStep
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat
)
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.conditions import ConditionLessThanOrEqualTo
from sagemaker.workflow.functions import JsonGet
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.model_metrics import MetricsSource, ModelMetrics

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


### Setup

In [18]:
# Setup SageMaker session and role
session = Session()
pipeline_session = PipelineSession()
role = get_execution_role()
region = session.boto_region_name
default_bucket = "sagemaker-us-east-1-807494057176"
model_package_group_name = "PMPredictiveMaintenanceModels"
model_s3_path = f"s3://{default_bucket}/predictive-maintenance-feature-store/output/xgb-2024-10-22-13-55-23/xgb-2024-10-22-13-55-23/output/model.tar.gz"

In [19]:
# Define pipeline parameters
processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)

training_instance_type = ParameterString(
    name="TrainingInstanceType",
    default_value="ml.m5.xlarge"
)

model_approval_status = ParameterString(
    name="ModelApprovalStatus",
    default_value="PendingManualApproval"
)

In [20]:
# Using bucket from the past notebook
input_data = ParameterString(
    name="InputData",
    default_value=f"s3://{default_bucket}/predictive-maintenance-model-monitor/datacapture"
)

quality_threshold = ParameterFloat(
    name="QualityThreshold",
    default_value=0.996
)

In [21]:
# Get container images
xgb_image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.7-1",
)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [22]:
# Create preprocessing step
processor = ScriptProcessor(
    image_uri=xgb_image_uri,
    command=["python3"],
    instance_type="ml.m5.xlarge",
    instance_count=1,
    base_job_name="pm-preprocess",
    role=role,
)

# Define preprocessing step with named outputs
step_process = ProcessingStep(
    name="PreprocessPMData",
    processor=processor,
    inputs=[
        ProcessingInput(
            source=input_data,
            destination="/opt/ml/processing/input"
        )
    ],
    outputs=[
        ProcessingOutput(
            output_name="train",
            source="/opt/ml/processing/train"
        ),
        ProcessingOutput(
            output_name="test",
            source="/opt/ml/processing/test"
        )
    ],
    code="code/preprocess.py"
)

In [23]:
# Create XGBoost estimator
xgb_estimator = sagemaker.estimator.Estimator(
    image_uri=xgb_image_uri,
    role=role,
    instance_count=1,
    instance_type=training_instance_type,
    output_path=f"s3://{default_bucket}/predictive-maintenance/models",
    sagemaker_session=pipeline_session,
)

# Set hyperparameters
xgb_estimator.set_hyperparameters(
    objective="binary:logistic",
    num_round=50,
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.7
)

In [24]:
step_train = TrainingStep(
    name="TrainPMModel",
    estimator=xgb_estimator,
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[0].S3Output.S3Uri,
            content_type="text/csv"
        )
    }
)

# Create evaluation step
evaluation_report = PropertyFile(
    name="EvaluationReport",
    output_name="evaluation",
    path="evaluation.json"
)

step_eval = ProcessingStep(
    name="EvaluatePMModel",
    processor=processor,
    inputs=[
        ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model"
        ),
        ProcessingInput(
            source=step_process.properties.ProcessingOutputConfig.Outputs[1].S3Output.S3Uri,
            destination="/opt/ml/processing/test"
        )
    ],
    outputs=[
        ProcessingOutput(
            output_name="evaluation",
            source="/opt/ml/processing/evaluation"
        )
    ],
    code="code/evaluate.py",
    property_files=[evaluation_report]
)

In [25]:
from sagemaker.workflow.functions import Join

# Create model metrics matching your monitoring metrics
model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=Join(
            on="/",
            values=[
                step_eval.properties.ProcessingOutputConfig.Outputs[0].S3Output.S3Uri,
                "evaluation.json"
            ]
        ),
        content_type="application/json"
    )
)

# Create register model step
step_register = RegisterModel(
    name="RegisterPMModel",
    estimator=xgb_estimator,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics,
)

# Create condition step
cond_lte = ConditionLessThanOrEqualTo(
    left=JsonGet(
        step_name=step_eval.name,
        property_file=evaluation_report,
        json_path="binary_classification_metrics.f2.value"
    ),
    right=quality_threshold
)

step_cond = ConditionStep(
    name="CheckModelQuality",
    conditions=[cond_lte],
    if_steps=[step_register],
    else_steps=[]
)

In [ ]:
from sagemaker.workflow.pipeline_definition_config import PipelineDefinitionConfig
# pipeline definition config
pipeline_definition_config = PipelineDefinitionConfig(
    use_custom_job_prefix=True
)

# make the pipeline
pipeline = Pipeline(
    name="PMPredictiveMaintenancePipeline",
    parameters=[
        processing_instance_count,
        training_instance_type,
        model_approval_status,
        input_data,
        quality_threshold
    ],
    steps=[step_process, step_train, step_eval, step_cond]
)

pipeline.upsert(
    role_arn=role,
    description="CI/CD pipeline for predictive maintenance model with quality monitoring"
)

## Start the pipeline execution with initial model
Code should run if you have access to the bucket

In [ ]:
execution = pipeline.start(
    parameters={
        "ProcessingInstanceCount": 1,
        "TrainingInstanceType": "ml.m5.xlarge",
        "ModelApprovalStatus": "PendingManualApproval",
        "QualityThreshold": 0.996  # initial F2 score threshold
    }
)

execution.wait()

### Analyze execution

In [ ]:
# Get detailed execution information
execution_steps = execution.list_steps()
print("Pipeline execution steps and their status:")
for step in execution_steps:
    print(f"\nStep: {step['StepName']}")
    print(f"Status: {step['StepStatus']}")
    if step['StepStatus'] == 'Failed':
        print(f"Failure Reason: {step.get('FailureReason', 'No failure reason provided')}")
        
# Get the full execution details
execution_details = execution.describe()
print("\nFull execution details:")
print(f"Status: {execution_details['PipelineExecutionStatus']}")
print(f"Failure Reason: {execution_details.get('FailureReason', 'No failure reason provided')}")

# List the CloudWatch log groups
logs_client = boto3.client('logs')
log_groups = logs_client.describe_log_groups(
    logGroupNamePrefix='/aws/sagemaker/'
)

print("\nAvailable CloudWatch Log Groups:")
for log_group in log_groups['logGroups']:
    print(log_group['logGroupName'])

## Eval Model

In [ ]:
print("Initial model evaluation results:")
initial_evaluation_json = json.loads(
    sagemaker.s3.S3Downloader.read_file(
        "{}/evaluation.json".format(
            step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
        )
    )
)
print("MSE:", initial_evaluation_json['regression_metrics']['mse']['value'])
print("Accuracy:", initial_evaluation_json['binary_classification_metrics']['accuracy']['value'])

## Will implement model improvement and auto deployment here

#### Delete all pipelines on account

In [ ]:
# Initialize the SageMaker client
sagemaker_client = boto3.client('sagemaker')

# List all pipelines in your account
pipelines = sagemaker_client.list_pipelines()

# Loop through all pipelines and delete each one
for pipeline in pipelines['PipelineSummaries']:
    pipeline_name = pipeline['PipelineName']
    print(f"Deleting pipeline: {pipeline_name}")
    
    # Delete the pipeline
    sagemaker_client.delete_pipeline(PipelineName=pipeline_name)

print("All pipelines deleted successfully.")